In [211]:
# Import the data
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder

df = pd.read_csv('dataResource/valentine_gift_large_dataset.csv')


In [212]:
# merge two samilar value to one vlaue
def mergeData(column_name, value_one, value_two):
    df[column_name].replace({value_one: value_two}, inplace=True)

In [ ]:
# encode column's value
def hot_encode(df):
       categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
       encoder = OneHotEncoder(sparse_output=False)
       one_hot_encoded = encoder.fit_transform(df[categorical_columns])

       one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
       df_encoded = pd.concat([df, one_hot_df], axis=1)
       df_encoded = df_encoded.drop(categorical_columns, axis=1)

       return df_encoded

In [ ]:
#  odeinal encode y values
def ordinal_encode(df):
    encoder = OrdinalEncoder(categories=[df['Best_Gift'].unique()])
    df['Best_Gift_Encoded'] = encoder.fit_transform(df[['Best_Gift']])
    print(df[['Best_Gift', 'Best_Gift_Encoded']].head())
    return df['Best_Gift_Encoded']
          


In [ ]:
def clean_data(df):
    
    # drop unsless columns
    df = df.drop(columns = ['Special_Note', 'Gift_Purpose', 'Past_Gift_Reaction', 'Delivery_Method', 'Gift_Wrapping_Preference'])

    # combine the samiliar values for few columns
    mergeData("Recipient_Gender", 'Non-Binary', 'Other')
    mergeData("Preference", "Sentimental", "Romantic")
    mergeData("Personal_Interest", "Sports", "Fitness")
    mergeData("Personal_Interest", "Gaming", "Technology")
    mergeData("Best_Gift", "Handmade Gift", "Personalized Gift")
    mergeData("Best_Gift", "Chocolates", "Chocolates & Flowers")
    mergeData("Best_Gift", "Flowers", "Chocolates & Flowers")
    mergeData("Best_Gift", "Jewelry", "Fashion Accessories")


    # split X and y
    # use hot encoding to encode some columns in x 
    X = df.drop(columns=['Best_Gift'])
    X = hot_encode (X)

    # use OrdinalEncoder to encode "Best_Gift" column and get y
    y = ordinal_encode(df)

    return X, y


X, y = clean_data(df)

X.head()






  




              Best_Gift  Best_Gift_Encoded
0               Perfume                0.0
1   Fashion Accessories                1.0
2  Chocolates & Flowers                2.0
3   Fashion Accessories                1.0
4  Subscription Service                3.0
y data: 0       0.0
1       1.0
2       2.0
3       1.0
4       3.0
       ... 
1995    5.0
1996    3.0
1997    3.0
1998    1.0
1999    6.0
Name: Best_Gift_Encoded, Length: 2000, dtype: float64


,Recipient_Age,Recipient_Gender_Female,Recipient_Gender_Male,Recipient_Gender_Other,Relationship_Status_Complicated,Relationship_Status_Dating,Relationship_Status_Engaged,Relationship_Status_Married,Relationship_Status_Single,Budget_High ($100-$500),...,Personal_Interest_Cooking,Personal_Interest_Fashion,Personal_Interest_Fitness,Personal_Interest_Music,Personal_Interest_Reading,Personal_Interest_Technology,Personal_Interest_Travel,Gift_Popularity_Moderately Popular,Gift_Popularity_Rare Choice,Gift_Popularity_Very Popular
0,29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,34,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,56,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,50,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,53,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
